In [1]:
import numpy as np
import cv2
from cv2 import dnn
from typing import Tuple

In [2]:
# Model file paths
proto_file: str = 'C:\\Users\\user\\Desktop\\ColorNet\\colorization_deploy_v2.prototxt'
model_file: str = 'C:\\Users\\user\\Desktop\\ColorNet\\colorization_release_v2.caffemodel'
hull_pts: str = 'C:\\Users\\user\\Desktop\\ColorNet\\pts_in_hull.npy'
img_path: str = 'C:\\Users\\user\\Desktop\\ColorNet\\Verm-sunset-clouds-Devils-Tower-4569.jpg'

In [3]:
# Reading the model params
net = dnn.readNetFromCaffe(proto_file, model_file)
kernel: np.ndarray = np.load(hull_pts)

In [4]:
# Reading and preprocessing image
img: np.ndarray = cv2.imread(img_path)
scaled: np.ndarray = img.astype("float32") / 255.0
lab_img: np.ndarray = cv2.cvtColor(scaled, cv2.COLOR_BGR2LAB)

In [5]:
# Add the cluster centers as 1x1 convolutions to the model
class8: int = net.getLayerId("class8_ab")
conv8: int = net.getLayerId("conv8_313_rh")
pts: np.ndarray = kernel.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]

In [6]:
# Resizing the image for the network 
resized: np.ndarray = cv2.resize(lab_img, (224, 224))

# Splitting the L channel 
L: np.ndarray = cv2.split(resized)[0]

# Mean subtraction
L -= 50

In [7]:
# Predicting the ab channels from the input L channel
net.setInput(cv2.dnn.blobFromImage(L))
ab_channel: np.ndarray = net.forward()[0, :, :, :].transpose((1, 2, 0)) 

# Resize the predicted 'ab' volume to the same dimensions as our input image
ab_channel: np.ndarray = cv2.resize(ab_channel, (img.shape[1], img.shape[0]))

In [8]:
# Taking the L channel from the image
L: np.ndarray = cv2.split(lab_img)[0]
# Join the L channel with predicted ab channel
colorized: np.ndarray = np.concatenate((L[:, :, np.newaxis], ab_channel), axis=2)

In [9]:
# Converting the image from Lab to BGR 
colorized: np.ndarray = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
colorized: np.ndarray = np.clip(colorized, 0, 1)
 
# Change the image to 0-255 range and convert it from float32 to int
colorized: np.ndarray = (255 * colorized).astype("uint8")

In [12]:
# Resizing the images and show them together
img: np.ndarray = cv2.resize(img,(640,640))
colorized: np.ndarray = cv2.resize(colorized,(640,640))
 
result: np.ndarray = cv2.hconcat([img,colorized])
 
cv2.imshow("Grayscale -> Colour", result)

output_path = 'colorized_img1.jpg'
cv2.imwrite(output_path, colorized)
input_path = 'original_img.jpg'
cv2.imwrite(input_path, img)
print(f"Colorized image saved to {output_path}")

cv2.waitKey(0)

Colorized image saved to colorized_img1.jpg


-1